# [SEMES] - 볼트 이상진단 - 예측

## Tensorflow

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

import tensorflow as tf
import numpy as np
import pandas as pd

# excel 파일을 불러와서 , 단위로 데이터를 분리해서 담는다
data = pd.read_csv('./price data.csv', sep=',')
# excel 파일의 데이터를 numpy로 변환하여 xy에 저장
xy = np.array(data, dtype=np.float32)

x_data = xy[:,1:-1]
y_data = xy[:, [-1]]

# X에 4개의 변인이 담길 수 있도록 만들어주고
X = tf.keras.Input(shape=[4], dtype=tf.float32)
# Y에 1개의 변인이 담길 수 있도록 만들어준다
Y = tf.keras.Input(shape=[1], dtype=tf.float32)

# 가설 
hypothesis = tf.matmul(X,W) + b
# 비용함수
cost = tf.reduce_mean(tf.square(hypothesis - Y))
# 최적화 함수
optimizer = tf.optimizers.SGD(learning_rate=0.000005)

# 모델 생성
model = tf.keras.Model(inputs=[X, Y], outputs=[cost, hypothesis])
# 모델 컴파일
model.compile(optimizer=optimizer, loss='mse')
# 학습
model.fit(x=[x_data, y_data], y=[y_data, y_data], epochs=100001, verbose=2)

## Pytorch

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

# GPU 사용 가능 확인
device = 'cuda:5' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

# 데이터 불러오기
data = pd.read_csv('./price data.csv', sep=',')
xy = np.array(data, dtype=np.float32)
x_data = xy[:, 1:-1]
y_data = xy[:, [-1]]

# 모델 구성
model = nn.Linear(4, 1) # input_dim=4, output_dim=1

# 손실 함수 정의
criterion = nn.MSELoss()

# optimizer 정의
optimizer = torch.optim.SGD(model.parameters(), lr=0.000005)

# 학습 진행
for epoch in range(100001):
# numpy array를 Tensor로 변환
    inputs = torch.from_numpy(x_data)
    targets = torch.from_numpy(y_data)

    # optimizer 초기화
    optimizer.zero_grad()

    # forward 연산
    outputs = model(inputs)

    # 손실 계산
    loss = criterion(outputs, targets)

    # backward 연산 및 가중치 갱신
    loss.backward()
    optimizer.step()

    if epoch % 10000 == 0:
        print('#', epoch, "손실비용 : ", loss.item())

## 학습된 모델을 이용하여 예측하기

In [ ]:
import torch

# GPU 사용 가능 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))


# 모델 평가 모드로 변경

model = torch.load('선형회귀모델.pth') 
model.to(device)
model.eval()

# 4가지 변수를 입력받아 예측가격 출력하는 코드
a = float(input().strip())
b = float(input().strip())
c = float(input().strip())
d = float(input().strip())

# 예측할 입력 데이터 정의
inputs = torch.tensor([[a,b,c,d]], dtype=torch.float32).to(device)

# 모델에 입력 데이터 전달하여 예측값 계산
predicted = model(inputs)

# 결과 출력
print('Predicted price : {:.4f}'.format(predicted.item()))